# TigerRAT and TigerLoader Time! 

A number of entities have observed the use of TigerRAT and its loader since at least 2020. Some great reporting has come out of: 
 - [ThreatRay](https://threatray.com/blog/establishing-the-tigerrat-and-tigerdownloader-malware-families/)
 - [Kaspersky](https://securelist.com/andariel-evolves-to-target-south-korea-with-ransomware/102811/)
 - [MalwareBytes](https://blog.malwarebytes.com/threat-intelligence/2021/04/lazarus-apt-conceals-malicious-code-within-bmp-file-to-drop-its-rat/)
 - [KrCERT](https://www.krcert.or.kr/filedownload.do?attach_file_seq=3277&attach_file_id=EpF3277.pdf)
 

What most interested me from a YARA perspective was the payload storage in the loader. Storing the payload in different portions of the PE was neat to observe, and provided a lot of samples for us to tinker with

![](https://threatray.com/wp-content/uploads/2021/12/figure6-1-1030x436.png)

Effectively the embedded payload is stored unto itself in either the last section, a resource, or the overlay. They follow the same structure (sorry you're not getting C Structs cause I don't know em well enough to not butcher them!): 
 - Length of the payload
 - 16 byte XOR key
 - null byte
 - base64-encoded payload
![](https://threatray.com/wp-content/uploads/2021/12/figure7-1-1030x400.png)

As usual Seongsu Park has some great images to highlight this (note there is a similar structure used by the second stage and third stages to load the next payload)

![](https://media.kasperskycontenthub.com/wp-content/uploads/sites/43/2021/06/15095212/Andariel_delivered_ransomware_03.png)

Lets take a gander at one of these samples! 

In [3]:
!emit /Users/glesnewich/Projects/Malware/TA430/tiger_rat_packed.bin | pemeta

{
    "Header": {
        "Machine": "AMD64",
        "Subsystem": "Windows GUI",
        "MinimumOS": "Windows Vista",
        "RICH": [
            {
                "Counter": 127,
                "Encoded": "00e05146",
                "Library": "UTC1800_C",
                "Product": "Visual Studio 2013 12.0"
            },
            {
                "Counter": 12,
                "Encoded": "00df5146",
                "Library": "MASM1200",
                "Product": "Visual Studio 2013 12.0"
            },
            {
                "Counter": 47,
                "Encoded": "00e15146",
                "Library": "UTC1800_CPP",
                "Product": "Visual Studio 2013 12.0"
            },
            {
                "Counter": 9,
                "Encoded": "00cbffdd",
                "Library": "IMPLIB1100",
                "Product": "Visual Studio 2012 11.0"
            },
            {
                "Counter": 162,
                "Encoded": "00010000",
       

In [4]:
!emit /Users/glesnewich/Projects/Malware/TA430/tiger_rat_packed.bin | vsect -l

.text
.rdata
.data
.pdata
.rsrc
.reloc
data


That last section, `data` looks sus - lets crack that section open

In [15]:
!emit /Users/glesnewich/Projects/Malware/TA430/tiger_rat_packed.bin | vsect [ | sorted size | pick 0:1 | peek

-------------------------------------------------------------------------------
00.337 MB; 73.58% entropy; data
  offset = 0x20000
    path = data
-------------------------------------------------------------------------------
00000000: AC 22 05 00 21 78 6B 24 63 29 46 23 34 40 32 67 38  ."..!xk$c)F#4@2g8
00000011: 64 4F 63 62 43 4C 37 4A 47 41 70 52 69 4D 77 51 44  dOcbCL7JGApRiMwQD
00000022: 4A 6E 78 35 74 50 49 63 42 72 4A 47 4D 70 52 69 4D  Jnx5tPIcBrJGMpRiM
00000033: 30 41 44 4A 6E 4F 47 52 50 49 58 68 72 4A 47 4D 70  0ADJnOGRPIXhrJGMp
00000044: 52 69 4D 30 51 44 4A 6E 4F 47 52 50 49 58 68 72 4A  RiM0QDJnOGRPIXhrJ
00000055: 47 4D 70 52 69 4D 30 51 44 4A 6E 4F 47 52 50 32 58  GMpRiM0QDJnOGRP2X
00000066: 68 72 4A 47 30 32 2F 43 30 30 39 44 75 71 47 64 78  hrJG02/C009DuqGdx
00000077: 4F 62 62 56 4B 63 41 74 41 4E 51 4E 45 4D 6C 30 41  ObbVKcAtANQNEMl0A
00000088: 53 67 55 69 41 52 73 4B 53 67 31 47 4D 67 4E 57 4A  SgUiARsKSg1GMgNWJ
00000099: 52 49 56 54 51 70 76 53 42 5A 4C 59 43 78 3

Ah ha! jackpot! Lets decode it to make sure I'm not off the mark here

In [3]:
!emit /Users/glesnewich/Projects/Malware/TA430/tiger_rat_packed.bin| vsect [ | sorted size \
   | pick 0:1 | snip 4: \
   | put key x::16 | b64 | xor key | peek

-------------------------------------------------------------------------------
00.252 MB; 97.33% entropy; MS-DOS executable, MZ for MS-DOS
     key = !xk$c)F#4@2g8dOc
  offset = 0x20000
    path = data
-------------------------------------------------------------------------------
00000000: 4D 5A 90 00 03 00 00 00 04 00 00 00 FF FF 00 42 E1  MZ.............B.
00000011: 13 4F 47 4A 6F 65 17 34 72 55 5F 5C 2B 6E 1B 4A 5C  .OGJoe.4rU_\+n.J\
00000022: 08 0D 25 0A 72 63 06 27 0A 03 77 45 37 08 05 1B 42  ..%.rc.'..wE7...B
00000033: 62 40 1D 06 11 53 78 56 28 E1 1C 24 47 4C 4E 97 09  b@...SxV(..$GLN..
00000044: 57 DD 7D 89 2D 9C 7C 0A 8D 2E 3F 68 61 4D 68 60 51  W.}.-.|...?haMh`Q
00000055: 74 46 69 31 62 33 7C 32 2E 42 25 13 7B 3D 01 71 3C  tFi1b3|2.B%.{=.q<
00000066: 0B 29 5B 08 24 2C 58 48 35 05 6F 23 4F 73 2A 1C 2F  .)[.$,XH5.o#Os*./
00000077: 66 31 38 59 43 5B 4D 09 40 1C 45 CB 9F 16 51 F5 8D  f18YC[M.@.E...Q..
00000088: 01 37 EA 8B 5C 3E 90 D8 F1 A9 21 93 4B 7A 9B D4 8C  .7..\>....!.Kz.

Okay there's at least the makings of an MZ header in there. This is very likely user error for not fully decoding properly :) 

Helpfully, ThreatRay published a series of helper scripts, including an [unpacker](https://github.com/threatray/tigerrat/blob/main/scripts/tigerlib/unpacker.py) which I tried to mimick in the above scenario. But thats just for the last section - what about overlays or resources? 

In [17]:
!emit /Users/glesnewich/Projects/Malware/TA430/TigerRAT/Packed/* [ | peoverlay | peek 

-------------------------------------------------------------------------------
00.048 kB; 33.25% entropy; data
-------------------------------------------------------------------------------
0000: 54 00 75 00 65 00 20 00 53 00 65 00 70 00 20 00 32 00  T.u.e...S.e.p...2.
0012: 32 00 20 00 30 00 39 00 3A 00 35 00 32 00 3A 00 35 00  2...0.9.:.5.2.:.5.
0024: 36 00 20 00 32 00 30 00 32 00 30 00                    6...2.0.2.0.
-------------------------------------------------------------------------------
00.048 kB; 35.43% entropy; data
-------------------------------------------------------------------------------
0000: 54 00 75 00 65 00 20 00 4A 00 61 00 6E 00 20 00 31 00  T.u.e...J.a.n...1.
0012: 39 00 20 00 31 00 35 00 3A 00 31 00 35 00 3A 00 34 00  9...1.5.:.1.5.:.4.
0024: 37 00 20 00 32 00 30 00 32 00 31 00                    7...2.0.2.1.
-------------------------------------------------------------------------------
00.048 kB; 33.77% entropy; data
------------------------------------

-------------------------------------------------------------------------------
00.337 MB; 72.88% entropy; data
-------------------------------------------------------------------------------
00000000: 58 25 05 00 77 4E 6D 48 4B 33 4A 39 32 45 5E 4B 45  X%..wNmHK3J92E^KE
00000011: 34 79 6A 00 4F 68 54 39 53 45 67 7A 53 6A 6B 32 52  4yj.OhT9SEgzSjk2R
00000022: 56 35 4C 75 73 74 35 64 2F 5A 74 53 45 73 7A 53 6A  V5Lust5d/ZtSEszSj
00000033: 6B 79 42 56 35 4C 52 54 52 35 64 30 35 74 53 45 73  kyBV5LRTR5d05tSEs
00000044: 7A 53 6A 6B 79 52 56 35 4C 52 54 52 35 64 30 35 74  zSjkyRV5LRTR5d05t
00000055: 53 45 73 7A 53 6A 6B 79 52 56 35 4C 52 54 52 35 6E  SEszSjkyRV5LRTR5n
00000066: 30 35 74 53 45 55 73 38 44 63 79 38 56 65 47 5A 49  05tSEUs8Dcy8VeGZI
00000077: 78 34 4F 34 4E 4D 48 43 4E 61 4F 52 6C 43 4E 7A 45  x4O4NMHCNaORlCNzE
00000088: 73 4E 31 55 55 56 79 30 4D 4A 69 56 63 50 68 6C 51  sN1UUVy0MJiVcPhlQ
00000099: 49 48 34 35 4D 46 70 5A 48 69 42 4E 44 41 52 67 61  IH45MFpZHiBNDARga
--------

In [5]:
!emit /Users/glesnewich/Projects/Malware/TA430/tiger_rat_packed.bin| vsect [ | sorted size \
   | pick 0:1 | snip 0x52279: | peek -l 20
                                                                        

-------------------------------------------------------------------------------
00.391 kB; 17.54% entropy; ASCII text, with no line terminators
  offset = 0x20000
    path = data
-------------------------------------------------------------------------------
0000: 47 4D 70 52 69 4D 30 51 44 4A 6E 4F 47 52 50 49 58 68  GMpRiM0QDJnOGRPIXh
0012: 72 4A 47 4D 70 52 69 4D 30 51 44 4A 6E 4F 47 52 50 49  rJGMpRiM0QDJnOGRPI
0024: 58 68 72 4A 47 4D 70 52 69 4D 30 51 44 4A 6E 4F 47 52  XhrJGMpRiM0QDJnOGR
0036: 50 49 58 68 72 4A 47 4D 70 52 69 4D 30 51 44 49 3D 00  PIXhrJGMpRiM0QDI=.
0048: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ..................
....: ///////////////// repeats 14 times //////////////////  //////////////////
0156: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ..................
-------------------------------------------------------------------------------
��������


So can we take that payload length value from the front of the payload, and subtract the length of the key + 4 byte length marker, and find similar payloads? 

I was hoping to find a nice and tidy way to look for these things, and validate that the first two bytes of the base64 decoded value, xor'd with the first 2 bytes of the key, would be `MZ` but I haven't figured out a nice way in YARA to do that. So hunting for the payload lengths will have to do! 

Something I learned during this process was the importance of PADDING inside of PE sections, but not individual resources or overlays? Why? Because the PE sections get mapped into memory, and tl;dr, things are made easier if their padding rounds up to the next 0x100 size. So we have to fudge some of our calculations:

```
import "pe"

rule APT_NK_TA430_TigerLoader_Payload_LastSection
{

condition:
	uint32(pe.sections[pe.number_of_sections-1].raw_data_offset) != 0x0 and
	uint32(pe.sections[pe.number_of_sections-1].raw_data_offset) == 
        (pe.sections[pe.number_of_sections-1].virtual_size - (0x14))
}


rule APT_NK_TA430_TigerLoader_Payload_Overlay
{

condition:
	uint32(pe.overlay.offset) == (pe.overlay.size - 0x17) or
    uint32(pe.overlay.offset) == (pe.overlay.size - 70 ) or
    uint32(pe.overlay.offset) == (pe.overlay.size - 21 )
}
```

So what are we doing here? Well, we're attempting to look for the byte marker at the start of the overlay or the last section as length field, and compare that integer to the integer of the size of the entire chunk we are assessing, minus the set length of the marker + xor key. 

While this works decently well on the section side, its a nightmare on the Overlay side because sometimes the payload has an extra null byte at the end; sometimes it doesn't; sometimes it has one of those date-time fields appended to the end of them as seen in the output a few cells above. 

Okay okay okay maybe I'm drastly over thinking this! Lets try another tactic - using the console module to mine out the import hashes and rich header hashes:

```
import "console"
import "pe"
import "hash"

rule imphash_printer
{

	condition:
			console.log("pe.imphash() == \"\" ", pe.imphash() )
}


rule richheader_printer
{
	condition:
			 console.log("hash.md5(pe.rich_signature.clear_data) == ", hash.md5(pe.rich_signature.clear_data))
}

```

In [18]:
!yara /Users/glesnewich/Desktop/console.yar /Users/glesnewich/Projects/Malware/TA430/TigerRAT/Packed/ -r 

pe.imphash() == "" fb892be2f8e28d3a116b556db68cdbb0
pe.imphash() == "" 7d3a5cb7a3140197f0e2e6f87d19764e
hash.md5(pe.rich_signature.clear_data) == 2cd8928b02b7b03341e893527f303378
hash.md5(pe.rich_signature.clear_data) == dd3c533b17d82f89a086b99a0a67908c
pe.imphash() == "" fb892be2f8e28d3a116b556db68cdbb0
imphash_printer /Users/glesnewich/Projects/Malware/TA430/TigerRAT/Packed//d0fa0bfef8b199a42f4f33145274576e5a7edeb5522fb342af41fdc16e9021e2
richheader_printer /Users/glesnewich/Projects/Malware/TA430/TigerRAT/Packed//d0fa0bfef8b199a42f4f33145274576e5a7edeb5522fb342af41fdc16e9021e2
hash.md5(pe.rich_signature.clear_data) == 2cd8928b02b7b03341e893527f303378
imphash_printer /Users/glesnewich/Projects/Malware/TA430/TigerRAT/Packed//9137e886e414b12581852b96a1d90ee875053f16b79be57694df9f93f3ead506
richheader_printer /Users/glesnewich/Projects/Malware/TA430/TigerRAT/Packed//9137e886e414b12581852b96a1d90ee875053f16b79be57694df9f93f3ead506
pe.imphash() == "" 7d3a5cb7a3140197f0e2e6f87d19764e
hash.

Quick clean up with sort + uniq 

In [ ]:
!yara /Users/glesnewich/Desktop/console.yar /Users/glesnewich/Projects/Malware/TA430/TigerRAT/Packed/ -r  \
| sort | uniq -c | sort -rn | less

  11 hash.md5(pe.rich_signature.clear_data) == 9b9c6f8e3e374c8f712c0eb37cd63e16
   9 pe.imphash() == "" fb892be2f8e28d3a116b556db68cdbb0
   9 hash.md5(pe.rich_signature.clear_data) == 2cd8928b02b7b03341e893527f303378
   5 pe.imphash() == "" 7d3a5cb7a3140197f0e2e6f87d19764e
   5 hash.md5(pe.rich_signature.clear_data) == dd3c533b17d82f89a086b99a0a67908c
   2 pe.imphash() == "" 0e15d4b92e4de8ce71e3c186bb853030
   1 richheader_printer /Users/glesnewich/Projects/Malware/TA430/TigerRAT/Packed //f62adc678eaadc019277640e6695143a45336c2f91019f5d9308812db1d07285
   1 richheader_printer /Users/glesnewich/Projects/Malware/TA430/TigerRAT/Packed //f4765f7b089d99b1cdcebf3ad7ba7e3e23ce411deab29b7afd782b23352e698f
   1 richheader_printer /Users/glesnewich/Projects/Malware/TA430/TigerRAT/Packed //f13aff9e1192c081c012f974b29bf60487385eed644d506d7f82b3538c2b035f
   1 richheader_printer /Users/glesnewich/Projects/Malware/TA430/TigerRAT/Packed //ed5fbefd61a72ec9f8a5ebd7fa7bcd632ec55f04bdd4a4e24686edccb0

some pretty common rich headers and import hashes:
```
     11 hash.md5(pe.rich_signature.clear_data) == 9b9c6f8e3e374c8f712c0eb37cd63e16
     9 pe.imphash() == "" fb892be2f8e28d3a116b556db68cdbb0
     9 hash.md5(pe.rich_signature.clear_data) == 2cd8928b02b7b03341e893527f303378
```

In [7]:
!yara /Users/glesnewich/Desktop/APT_NK_TA430_Tiger.yar /Users/glesnewich/Projects/Malware/ -r | sort

APT_NK_TA430_TigerLoader_ImportHash /Users/glesnewich/Projects/Malware//TA430/TigerRAT/Packed/0dc3f66f4af3250f56a32f8e1b9e772c514f74718358d19c195e3950d370ea01
APT_NK_TA430_TigerLoader_ImportHash /Users/glesnewich/Projects/Malware//TA430/TigerRAT/Packed/1892b72c053ab48edae8305ef449f2b5391921efea8b1d7c37d6d29f59edc92e
APT_NK_TA430_TigerLoader_ImportHash /Users/glesnewich/Projects/Malware//TA430/TigerRAT/Packed/2f53109e01c431c1c1acec667adee07cf907cdc4d36429022f915654c9b7113b
APT_NK_TA430_TigerLoader_ImportHash /Users/glesnewich/Projects/Malware//TA430/TigerRAT/Packed/7d7dc8125a26d9515d90a66bfd20d609820197c879030cb932d39b1c2998e9d4
APT_NK_TA430_TigerLoader_ImportHash /Users/glesnewich/Projects/Malware//TA430/TigerRAT/Packed/87f389d8f3a63f0879aa9d9dfbbd2b2c9cf678b871b704a01b39e1eaa234020c
APT_NK_TA430_TigerLoader_ImportHash /Users/glesnewich/Projects/Malware//TA430/TigerRAT/Packed/d0fa0bfef8b199a42f4f33145274576e5a7edeb5522fb342af41fdc16e9021e2
APT_NK_TA430_TigerLoader_ImportHash /Users/gle

If we wanted to tally the votes, we'll see that the lower hanging fruit of the rich headers catches more true positives (24) than the funky ones looking to use the decoding structure as rule logic. 

While this was largely an exercise in futility, I hope this highlights that sometimes the more simple the YARA rule, the better! But for the road, there is this strangely common function found at the entry point of both stages of TigerRAT... 

```


rule APT_NK_TA430_TigerLoader_CommonEntryFunc {
meta:
	author = "Greg Lesnewich and some secret sauce"
	date = "2023-01-05"
	version = "1.0"
	hash = "0996a8e5ec1a41645309e2ca395d3a6b766a7c52784c974c776f258c1b25a76c"
	hash = "0e447797aa20bff416073281adb09b73c15433ab855b5cdb2d883f8c2af9c414"
	hash = "1f8dcfaebbcd7e71c2872e0ba2fc6db81d651cf654a21d33c78eae6662e62392"
	hash = "4d03a981bed15a3bd91f36972d7391b39791c582bb2959a9be154a74bd64db31"
	hash = "4da0ac4c3f47f69c992abb5d6e9803348bf9f3c6028a7214dcabec9a2e729b99"
	hash = "6310cd9f8b6ae1fdc1b55fe190026a119f7ea526cd3fc22a215bda51c9c28214"
	hash = "69bac736f42e37302db7eca68b6fc138c3aa9a5c902c149e46cce8b42b172603"
	hash = "9137e886e414b12581852b96a1d90ee875053f16b79be57694df9f93f3ead506"
	hash = "b0d6aee39e988196fdc821895a1f1aa63d1c032ea880c26a15c857068f34bfd9"
	hash = "b59e8f44822ad6bc3b4067bfdfd1ad286b8ba76c1a3faff82a3feb7bdf96b9c5"
	hash = "d231f3b6d6e4c56cb7f149cbc0178f7b80448c24f14dced5a864015512b0ba1f"
	hash = "d26987b705f537b10a11fb9913d0acc0218a0c0ae5f27e6f821d6d987b1cd4c7"
	hash = "da787cf1f4fd829dd4a7637bec392438b793c5f9c920560197545d20b58691af"
	hash = "ed5fbefd61a72ec9f8a5ebd7fa7bcd632ec55f04bdd4a4e24686edccb0268e05"
	hash = "f13aff9e1192c081c012f974b29bf60487385eed644d506d7f82b3538c2b035f"
	hash = "f4765f7b089d99b1cdcebf3ad7ba7e3e23ce411deab29b7afd782b23352e698f"
    reference = "https://threatray.com/blog/establishing-the-tigerrat-and-tigerdownloader-malware-families/"
	reference = "https://www.krcert.or.kr/filedownload.do?attach_file_seq=3277&attach_file_id=EpF3277.pdf"
	reference = "https://blog.malwarebytes.com/threat-intelligence/2021/04/lazarus-apt-conceals-malicious-code-within-bmp-file-to-drop-its-rat/"
	reference = "https://securelist.com/andariel-evolves-to-target-south-korea-with-ransomware/102811/"

strings:
$entry = { 48 83 EC 28 E8 ?? ?? ?? ?? 48 83 C4 28 E9 ?? ?? ?? ?? ?? ?? 48 89 5C 24 10 48 89 74 24 18 57 48 83 EC 30 E8 ?? ?? ?? ?? 0F B7 F0 B9 02 ?? ?? ?? E8 ?? ?? ?? ?? B8 ?? ?? ?? ?? 48 ?? ?? ?? ?? ?? ?? 66 39 ?? ?? ?? ?? ?? 74 ?? 33 DB EB ?? 48 ?? ?? ?? ?? ?? ?? 48 03 C7 81 38 50 45 ?? ?? 75 ?? B9 ?? ?? ?? ?? 66 39 48 18 75 ?? 33 DB 83 B8 84 ?? ?? ?? 0E 76 ?? 39 98 F8 ?? ?? ?? 0F 95 C3 89 5C 24 40 E8 ?? ?? ?? ?? 85 C0 75 ?? 83 ?? ?? ?? ?? ?? ?? 75 ?? E8 ?? ?? ?? ?? B9 ?? ?? ?? ?? E8 ?? ?? ?? ?? B9 ?? ?? ?? ?? E8 ?? ?? ?? ?? E8 ?? ?? ?? ?? 85 C0 75 ?? 83 ?? ?? ?? ?? ?? ?? 75 ?? E8 ?? ?? ?? ?? B9 ?? ?? ?? ?? E8 ?? ?? ?? ?? B9 ?? ?? ?? ?? E8 ?? ?? ?? ?? E8 ?? ?? ?? ?? 90 E8 ?? ?? ?? ?? 85 C0 79 ?? B9 ?? ?? ?? ?? E8 ?? ?? ?? ?? FF ?? ?? ?? ?? ?? 48 ?? ?? ?? ?? ?? ?? E8 ?? ?? ?? ?? 48 ?? ?? ?? ?? ?? ?? E8 ?? ?? ?? ?? 85 C0 79 ?? B9 08 ?? ?? ?? E8 ?? ?? ?? ?? E8 ?? ?? ?? ?? 85 C0 79 ?? B9 09 ?? ?? ?? E8 ?? ?? ?? ?? B9 01 ?? ?? ?? E8 ?? ?? ?? ?? 85 C0 74 ?? 8B C8 E8 ?? ?? ?? ?? E8 ?? ?? ?? ?? 44 0F B7 CE 4C 8B C0 33 D2 48 8B CF E8 ?? ?? ?? ?? 8B F8 89 44 24 20 85 DB 75 ?? 8B C8 E8 ?? ?? ?? ?? E8 ?? ?? ?? ?? EB ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? 8B C7 48 8B 5C 24 48 48 8B 74 24 50 48 83 C4 30 5F C3 }
 /*
Disassembly take from entry0@0996a8e5ec1a41645309e2ca395d3a6b766a7c52784c974c776f258c1b25a76c

┌ entry0 ();
│           ; var int64_t var_20h @ rsp+0x20
│           ; var int64_t var_40h @ rsp+0x40
│           ; var int64_t var_10h @ rsp+0x48
│           ; var int64_t var_18h @ rsp+0x50
│           0x1400048b0      4883ec28       sub   rsp, 0x28
│           0x1400048b4      e83f480000     call  fcn.1400090f8
│           0x1400048b9      4883c428       add   rsp, 0x28
│       ┌─< 0x1400048bd      e902000000     jmp   0x1400048c4
        │   0x1400048c2      cc             int3
        │   0x1400048c3      cc             int3
│       │   ; CODE XREF from entry0 @ 0x1400048bd
│       └─> 0x1400048c4      48895c2410     mov   qword [var_10h], rbx
│           0x1400048c9      4889742418     mov   qword [var_18h], rsi
│           0x1400048ce      57             push  rdi
│           0x1400048cf      4883ec30       sub   rsp, 0x30
│           0x1400048d3      e8741b0000     call  fcn.14000644c
│           0x1400048d8      0fb7f0         movzx esi, ax
│           0x1400048db      b902000000     mov   ecx, 2               ; int64_t arg1
│           0x1400048e0      e8cb470000     call  fcn.1400090b0
│           0x1400048e5      b84d5a0000     mov   eax, 0x5a4d          ; 'MZ'
│           0x1400048ea      488d3d0fb7ff.  lea   rdi, [0x140000000]
│           0x1400048f1      66390508b7ff.  cmp   word [0x140000000], ax ; [0x140000000:2]=0xffff
│       ┌─< 0x1400048f8      7404           je    0x1400048fe
│     ┌┌──> 0x1400048fa      33db           xor   ebx, ebx
│    ┌────< 0x1400048fc      eb31           jmp   0x14000492f
│    │╎╎└─> 0x1400048fe      48630537b7ff.  movsxd rax, dword [0x14000003c] ; [0x14000003c:4]=-1
│    │╎╎    0x140004905      4803c7         add   rax, rdi
│    │╎╎    0x140004908      813850450000   cmp   dword [rax], 0x4550
│    │└───< 0x14000490e      75ea           jne   0x1400048fa
│    │ ╎    0x140004910      b90b020000     mov   ecx, 0x20b           ; 523
│    │ ╎    0x140004915      66394818       cmp   word [rax + 0x18], cx
│    │ └──< 0x140004919      75df           jne   0x1400048fa
│    │      0x14000491b      33db           xor   ebx, ebx
│    │      0x14000491d      83b884000000.  cmp   dword [rax + 0x84], 0xe
│    │  ┌─< 0x140004924      7609           jbe   0x14000492f
│    │  │   0x140004926      3998f8000000   cmp   dword [rax + 0xf8], ebx
│    │  │   0x14000492c      0f95c3         setne bl
│    │  │   ; CODE XREF from entry0 @ 0x1400048fc
│    └──└─> 0x14000492f      895c2440       mov   dword [var_40h], ebx
│           0x140004933      e828100000     call  fcn.140005960
│           0x140004938      85c0           test  eax, eax
│       ┌─< 0x14000493a      7522           jne   0x14000495e
│       │   0x14000493c      833de5af0100.  cmp   dword [0x14001f928], 1 ; [0x14001f928:4]=0
│      ┌──< 0x140004943      7505           jne   0x14000494a
│      ││   0x140004945      e8f21f0000     call  fcn.14000693c
│      └──> 0x14000494a      b91c000000     mov   ecx, 0x1c            ; 28 ; int64_t arg1
│       │   0x14000494f      e85c200000     call  fcn.1400069b0
│       │   0x140004954      b9ff000000     mov   ecx, 0xff            ; 255 ; int64_t arg1
│       │   0x140004959      e8aaf6ffff     call  fcn.140004008
│       └─> 0x14000495e      e801320000     call  fcn.140007b64
│           0x140004963      85c0           test  eax, eax
│       ┌─< 0x140004965      7522           jne   0x140004989
│       │   0x140004967      833dbaaf0100.  cmp   dword [0x14001f928], 1 ; [0x14001f928:4]=0
│      ┌──< 0x14000496e      7505           jne   0x140004975
│      ││   0x140004970      e8c71f0000     call  fcn.14000693c
│      └──> 0x140004975      b910000000     mov   ecx, 0x10            ; 16 ; int64_t arg1
│       │   0x14000497a      e831200000     call  fcn.1400069b0
│       │   0x14000497f      b9ff000000     mov   ecx, 0xff            ; 255 ; int64_t arg1
│       │   0x140004984      e87ff6ffff     call  fcn.140004008
│       └─> 0x140004989      e8f2160000     call  fcn.140006080
│           0x14000498e      90             nop
│           0x14000498f      e838390000     call  fcn.1400082cc
│           0x140004994      85c0           test  eax, eax
│       ┌─< 0x140004996      790a           jns   0x1400049a2
│       │   0x140004998      b91b000000     mov   ecx, 0x1b            ; 27 ; int64_t arg1
│       │   0x14000499d      e8ae000000     call  fcn.140004a50
│       └─> 0x1400049a2      ff1590c80000   call  qword [sym.imp.KERNEL32.dll_GetCommandLineA] ; [sym.imp.KERNEL32.dll_GetCommandLineA:8]=0x1befe reloc.KERNEL32.dll_GetCommandLineA ; LPSTR GetCommandLineA(void)
│           0x1400049a8      48890599eb01.  mov   qword [0x140023548], rax ; [0x140023548:8]=0
│           0x1400049af      e8f0470000     call  fcn.1400091a4
│           0x1400049b4      4889051da301.  mov   qword [0x14001ecd8], rax ; [0x14001ecd8:8]=0
│           0x1400049bb      e800430000     call  fcn.140008cc0
│           0x1400049c0      85c0           test  eax, eax
│       ┌─< 0x1400049c2      790a           jns   0x1400049ce
│       │   0x1400049c4      b908000000     mov   ecx, 8               ; int64_t arg1
│       │   0x1400049c9      e852f6ffff     call  fcn.140004020
│       └─> 0x1400049ce      e8a9450000     call  fcn.140008f7c
│           0x1400049d3      85c0           test  eax, eax
│       ┌─< 0x1400049d5      790a           jns   0x1400049e1
│       │   0x1400049d7      b909000000     mov   ecx, 9               ; int64_t arg1
│       │   0x1400049dc      e83ff6ffff     call  fcn.140004020
│       └─> 0x1400049e1      b901000000     mov   ecx, 1               ; int64_t arg1
│           0x1400049e6      e87df6ffff     call  fcn.140004068
│           0x1400049eb      85c0           test  eax, eax
│       ┌─< 0x1400049ed      7407           je    0x1400049f6
│       │   0x1400049ef      8bc8           mov   ecx, eax             ; int64_t arg1
│       │   0x1400049f1      e82af6ffff     call  fcn.140004020
│       └─> 0x1400049f6      e89d480000     call  fcn.140009298
│           0x1400049fb      440fb7ce       movzx r9d, si
│           0x1400049ff      4c8bc0         mov   r8, rax
│           0x140004a02      33d2           xor   edx, edx
│           0x140004a04      488bcf         mov   rcx, rdi             ; int64_t arg3
│           0x140004a07      e854d9ffff     call  fcn.140002360
│           0x140004a0c      8bf8           mov   edi, eax
│           0x140004a0e      89442420       mov   dword [var_20h], eax
│           0x140004a12      85db           test  ebx, ebx
│       ┌─< 0x140004a14      7507           jne   0x140004a1d
│       │   0x140004a16      8bc8           mov   ecx, eax             ; int64_t arg1
│       │   0x140004a18      e887f9ffff     call  fcn.1400043a4
│       └─> 0x140004a1d      e836f6ffff     call  fcn.140004058
│       ┌─< 0x140004a22      eb17           jmp   0x140004a3b
        │   0x140004a24      8bf8           mov   edi, eax
        │   0x140004a26      837c244000     cmp   dword [rsp + 0x40], 0
       ┌──< 0x140004a2b      7508           jne   0x140004a35
       ││   0x140004a2d      8bc8           mov   ecx, eax
       ││   0x140004a2f      e8ccf6ffff     call  fcn.140004100
       ││   0x140004a34      cc             int3
       └──> 0x140004a35      e80ef6ffff     call  fcn.140004048
        │   0x140004a3a      90             nop
│       │   ; CODE XREF from entry0 @ 0x140004a22
│       └─> 0x140004a3b      8bc7           mov   eax, edi
│           0x140004a3d      488b5c2448     mov   rbx, qword [var_10h]
│           0x140004a42      488b742450     mov   rsi, qword [var_18h]
│           0x140004a47      4883c430       add   rsp, 0x30
│           0x140004a4b      5f             pop   rdi
└           0x140004a4c      c3             ret

 */
condition:
	$entry at pe.entry_point
}
```